# Data pre-processing

Loading env variables:

In [ ]:
import os
import sys
from dotenv import load_dotenv

load_dotenv(override=True)
print('ENV variables loaded successfully!')

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Loading unified dataset:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lib.env_var_keys import EnvVarKeys
from lib.dataframe_helper import vdrl_count

unified_dataset_path = os.getenv(EnvVarKeys.UNIFIED_DATASET_PATH_KEY.value)
df_unified = pd.read_csv(unified_dataset_path, sep=',', low_memory=False)

print(f'Unified dataset shape: {df_unified.shape}')
vdrl_count(df_unified, 'df_unified')

## Removing and renaming columns/features manually:

In [ ]:
columns_to_keep = [
  'mc_cri_vdrl', # mc_cri_crianca
  'mc_get_alcool', # mc_get_gestacao
  'mc_get_fator_rh', # mc_get_gestacao
  'mc_get_fumo', # mc_get_gestacao
  'mc_get_gravidez_planejada', # mc_get_gestacao
  'mc_get_grupo_sanguineo', # mc_get_gestacao
  'mc_get_risco_gestacional', # mc_get_gestacao
  'mc_get_vacina_anti_tetanica', # mc_get_gestacao
  'mc_get_dt_incl', # mc_get_gestacao (it'll used to calculate 'idade')
  'mc_mul_chefe_familia', # mc_mul_mulher
  'mc_mul_data_nascimento', # mc_mul_mulher (it'll used to calculate 'idade')
  'mc_mul_est_civil', # mc_mul_mulher
  'mc_mul_nivel_inseguranca', # mc_mul_mulher
  'mc_mul_qtd_aborto', # mc_mul_mulher
  'mc_mul_qtd_filhos_vivos', # mc_mul_mulher
  'mc_mul_qtd_gest', # mc_mul_mulher
  'mc_mul_rec_inf_plan_fam', # mc_mul_mulher
  'mc_mul_tipo_const_casa', # mc_mul_mulher
  'mc_mul_renda_familiar', # mc_mul_mulher
  'mc_dae_escolaridade', # mc_dae_dado_evolutivo
  'mc_dae_mrd_lgd_red_esg', # mc_dae_dado_evolutivo
  'mc_dae_numero_res_domic', # mc_dae_dado_evolutivo
  'mc_dae_possui_arv_frut', # mc_dae_dado_evolutivo
  'mc_dae_possui_horta', # mc_dae_dado_evolutivo
  'mc_dae_rfa', # mc_dae_dado_evolutivo
  'mc_dae_sit_moradia', # mc_dae_dado_evolutivo
  'mc_dae_trat_agua_uso', # mc_dae_dado_evolutivo
]

df_unified = df_unified[columns_to_keep]

print(f'Unified dataframe only with columns selected manually: {df_unified.shape}')

Keep only VDRL positive and negative:

In [ ]:
# Only using VDRL positive and negative data
df_unified = df_unified[df_unified['mc_cri_vdrl'].notna()]
print(f'Shape after removing rows with missing VDRL: {df_unified.shape}')

Removing outliers:

In [ ]:
# Removendo outliers da data de nascimento
MIN_YEAR = 1960
MAX_YEAR = 2020
df_unified['ano_nascimento'] = df_unified.apply(
  lambda row: None if row['mc_mul_data_nascimento'] is None else pd.to_datetime(row['mc_mul_data_nascimento'], errors='coerce').year, axis=1
)
df_unified = df_unified[(df_unified['ano_nascimento'].between(MIN_YEAR, MAX_YEAR)) | (df_unified['ano_nascimento'].isna())]
df_unified = df_unified.drop(columns=['ano_nascimento'])
print(f"Shape after removing outliers of 'mc_mul_data_nascimento': {df_unified.shape}")

# Valor máximo para a renda familiar
MAX_DAE_RFA = 20_000
df_unified = df_unified[(df_unified['mc_dae_rfa'] <= MAX_DAE_RFA) | (df_unified['mc_dae_rfa'].isna())]
print(f"Shape after removing outliers of 'mc_dae_rfa': {df_unified.shape}")

MAX_NUMERO_RES_DOMIC = 20
df_unified = df_unified[(df_unified['mc_dae_numero_res_domic'] <= MAX_NUMERO_RES_DOMIC) | (df_unified['mc_dae_numero_res_domic'].isna())]
print(f"Shape after removing outliers of 'mc_dae_numero_res_domic': {df_unified.shape}")

vdrl_count(df_unified, 'df_unified')

Creating `idade` column and removing `mc_mul_data_nascimento` and `mc_get_dt_incl`:

In [ ]:
df_unified['idade'] = df_unified.apply(
    lambda row: None if row['mc_mul_data_nascimento'] is None else (pd.to_datetime(row['mc_get_dt_incl']).year - pd.to_datetime(row['mc_mul_data_nascimento'], errors='coerce').year), axis=1
)

# Remove 'mc_mul_data_nascimento' column
df_unified = df_unified.drop(columns=['mc_mul_data_nascimento', 'mc_get_dt_incl'], axis=1)

print(f'Shape: {df_unified.shape}')

Removing rows with `mc_mul_renda_familiar` empty (11 rows):

In [ ]:
df_unified = df_unified.dropna(subset=['mc_mul_renda_familiar'])
print(f'Shape: {df_unified.shape}')

vdrl_count(df_unified, 'df_unified')

## Count empty columns:

Creating a new column (`empty_count`) to count the total of empty columns in each row and selecting all rows with `mc_cri_vdrl == 0` (**VDRL True**), rows with `mc_cri_vdrl == 1 AND empt_count == 0` and rows with `mc_cri_vdrl == 1 AND empty_count == 1` until reaching the **VDRL True** rows count:

In [ ]:
df_unified['empty_count'] = df_unified.apply(
    lambda row: sum(row.isna()), axis=1)

df_unified['empty_columns'] = df_unified.apply(lambda row: ' | '.join(row[row.isna()].to_dict().keys()), axis=1)

df_vdrl_empty_count = df_unified[[
    'empty_count', 'mc_cri_vdrl']].value_counts().reset_index(name='counts')
df_vdrl_empty_count['cri_vdrl_bool'] = df_vdrl_empty_count['mc_cri_vdrl'].apply(
    lambda x: True if x == 0 else False)
df_vdrl_empty_count = df_vdrl_empty_count[['mc_cri_vdrl', 'cri_vdrl_bool', 'empty_count', 'counts']].sort_values(
    by=['cri_vdrl_bool', 'counts'], ascending=[True, False]).reset_index(drop=True)

df_vdrl_empty_count.style.apply(lambda x: ['background: #9c3d44' if (x.empty_count == 0 or x.cri_vdrl_bool == True) else '' for i in x], axis=1).apply(
    lambda x: ['background: green' if (x.empty_count == 1 and x.cri_vdrl_bool == False) else '' for i in x], axis=1)



Renaming columns:

In [ ]:
df_unified = df_unified.rename(columns={'mc_cri_vdrl': 'VDRL_RESULT',
'mc_get_alcool': 'CONS_ALCOHOL',
'mc_get_fator_rh': 'RH_FACTOR',
'mc_get_fumo': 'SMOKER',
'mc_get_gravidez_planejada': 'PLAN_PREGNANCY',
'mc_get_grupo_sanguineo': 'BLOOD_GROUP',
'mc_get_risco_gestacional': 'HAS_PREG_RISK',
'mc_get_vacina_anti_tetanica': 'TET_VACCINE',
'mc_mul_chefe_familia': 'IS_HEAD_FAMILY',
'mc_mul_est_civil': 'MARITAL_STATUS',
'mc_mul_nivel_inseguranca': 'FOOD_INSECURITY',
'mc_mul_qtd_aborto': 'NUM_ABORTIONS',
'mc_mul_qtd_filhos_vivos': 'NUM_LIV_CHILDREN',
'mc_mul_qtd_gest': 'NUM_PREGNANCIES',
'mc_mul_rec_inf_plan_fam': 'FAM_PLANNING',
'mc_mul_tipo_const_casa': 'TYPE_HOUSE',
'mc_mul_renda_familiar': 'HAS_FAM_INCOME',
'mc_dae_escolaridade': 'LEVEL_SCHOOLING',
'mc_dae_mrd_lgd_red_esg': 'CONN_SEWER_NET',
'mc_dae_numero_res_domic': 'NUM_RES_HOUSEHOLD',
'mc_dae_possui_arv_frut': 'HAS_FRU_TREE',
'mc_dae_possui_horta': 'HAS_VEG_GARDEN',
'mc_dae_rfa': 'FAM_INCOME',
'mc_dae_sit_moradia': 'HOUSING_STATUS',
'mc_dae_trat_agua_uso': 'WATER_TREATMENT',
'idade': 'AGE'})

Saving pre-processed data set, with columns `emtpy_count` and `empty_columns`:

In [ ]:
pre_processed_dataset_path = os.getenv(EnvVarKeys.PRE_PROCESSED_DATASET_PATH_KEY.value)
df_unified.to_csv(pre_processed_dataset_path, sep=',',  index=False)

print(f'Pre-processed dataset saved to: {pre_processed_dataset_path}')